In [22]:
from sympy import *
import numpy as np
import dimod
from dimod.reference.samplers import ExactSolver
import pandas as pd
init_printing(use_unicode=True)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [23]:
lattice = Matrix([[1, 2], [2, 2]])
lattice.det()

In [24]:
(d1, d2) = shape(lattice)

In [25]:
def calculate_gram_matrix(lattice):
    (d1, d2) = shape(lattice)
    return Matrix([[lattice.row(i)*lattice.row(j).T for i in range(0, d1)] for j in range(0,d2)])

In [26]:
G = calculate_gram_matrix(lattice)
G

⎡5  6⎤
⎢    ⎥
⎣6  8⎦

In [27]:
def max_num_qubits_binary(lattice):
    return int(np.ceil((3.0/2.0)*shape(lattice)[0]*np.log2(shape(lattice)[0])+shape(lattice)[0]+np.log2(float(np.absolute(lattice.det())))))

In [28]:
max_num_qubits_binary(lattice)

In [29]:
shape_qubits_binary = [num_qudits, dim_qudits] = [d1, max_num_qubits_binary(lattice)]
print(shape_qubits_binary)

[2, 6]


In [37]:
s = symbols('s:'+str(shape_qubits_binary[0]*shape_qubits_binary[1]))
s

In [31]:
num_qubits = len(s)
shape_qubits = shape_qubits_binary

In [32]:
f = []
for i in range(0,shape_qubits[0]):
    expr = s[i*shape_qubits[1]]
    for j in range(1, shape_qubits[1]):
        expr += (2**j)*s[i*shape_qubits[1]+j]
    expr -= 2**(shape_qubits[1]-1)
    f.append(expr)
f

In [33]:
H = 0
for i in range(0,shape_qubits[0]):
    for j in range(0,shape_qubits[0]):
        H += G[i,j]*f[i]*f[j]
H

In [34]:
replacements = [(s[i]**2, s[i]) for i in range(num_qubits)]
H = H.expand().subs(replacements)
H

In [35]:
Q = H.as_coefficients_dict()

C = 0
h = {}
J = {}
Q_ = iter(Q)
for i in range(0, len(Q)):
    key = next(Q_)
    if i == 0:
        C = Q[key]
    elif i > 0 and i < num_qubits + 1:
        h[str(key)] = Q[key]
    else:
        keys = str(key).split('*')
        J[(keys[0], keys[1])] = Q[key]

        
print('Constant:\n',C)
print('\nLinear Terms:\n',h)
print('\nQuadratic Terms:\n',J)

Constant:
 25600

Linear Terms:
 {'s11': -20480, 's5': -17408, 's10': -12288, 's4': -9984, 's9': -6656, 's3': -5312, 's8': -3456, 's2': -2736, 's7': -1760, 's1': -1388, 's6': -888, 's0': -699}

Quadratic Terms:
 {('s0', 's6'): 12, ('s0', 's1'): 20, ('s0', 's7'): 24, ('s1', 's6'): 24, ('s6', 's7'): 32, ('s0', 's2'): 40, ('s0', 's8'): 48, ('s1', 's7'): 48, ('s2', 's6'): 48, ('s6', 's8'): 64, ('s0', 's3'): 80, ('s1', 's2'): 80, ('s0', 's9'): 96, ('s1', 's8'): 96, ('s2', 's7'): 96, ('s3', 's6'): 96, ('s6', 's9'): 128, ('s7', 's8'): 128, ('s0', 's4'): 160, ('s1', 's3'): 160, ('s0', 's10'): 192, ('s1', 's9'): 192, ('s2', 's8'): 192, ('s3', 's7'): 192, ('s4', 's6'): 192, ('s10', 's6'): 256, ('s7', 's9'): 256, ('s0', 's5'): 320, ('s1', 's4'): 320, ('s2', 's3'): 320, ('s0', 's11'): 384, ('s1', 's10'): 384, ('s2', 's9'): 384, ('s3', 's8'): 384, ('s4', 's7'): 384, ('s5', 's6'): 384, ('s10', 's7'): 512, ('s11', 's6'): 512, ('s8', 's9'): 512, ('s1', 's5'): 640, ('s2', 's4'): 640, ('s1', 's11'): 768

In [36]:
bqm = dimod.BinaryQuadraticModel(h, J, C, 'BINARY')
sampler = ExactSolver()
response = sampler.sample(bqm)
print(response)

     s0 s1 s10 s11 s2 s3 s4 s5 s6 s7 s8 s9  energy num_oc.
2     0  0   0   1  0  0  0  1  0  0  0  0     0.0       1
2459  1  0   1   0  0  0  0  1  1  1  1  1     1.0       1
2865  1  1   0   1  1  1  1  0  1  0  0  0     1.0       1
612   0  1   1   0  0  0  0  1  0  1  1  1     4.0       1
718   0  1   0   1  1  1  1  0  0  1  0  0     4.0       1
1230  0  1   0   1  1  1  1  0  1  0  0  0     4.0       1
1435  0  1   1   0  0  0  0  1  1  1  1  1     4.0       1
3278  1  1   0   1  1  1  1  0  0  0  0  0     5.0       1
3483  1  1   1   0  0  0  0  1  0  1  1  1     5.0       1
3790  1  0   0   1  1  1  1  0  0  1  0  0     5.0       1
4093  1  0   0   1  0  0  0  1  0  0  0  0     5.0       1
1636  0  0   1   0  0  0  0  1  1  1  1  1     8.0       1
1742  0  0   0   1  1  1  1  0  1  1  0  0     8.0       1
1892  0  0   1   0  1  0  0  1  1  0  1  1     8.0       1
2045  0  0   0   1  0  0  0  1  1  0  0  0     8.0       1
2353  1  0   0   1  1  1  1  0  1  1  0  0     9.0      

In [16]:
E = lambdify([s],H,'numpy')
name = []
Energy = []

data = {'0 1 2 3 4 5 6 7 8 9':name, 'Energy':Energy}
df = pd.DataFrame(data=data)
df.sort_values('Energy')

,0 1 2 3 4 5 6 7 8 9,Energy
